In [ ]:
!pip install pytorch_lightning

In [ ]:
!pip install comet_ml

In [ ]:
# used fo TPU
# Uncomment if you want to use TPU
# import collections
# from datetime import datetime, timedelta
# import os
# import requests
# import threading

# _VersionConfig = collections.namedtuple('_VersionConfig', 'wheels,server')
# VERSION = "xrt==1.15.0"  #@param ["xrt==1.15.0", "torch_xla==nightly"]
# CONFIG = {
#     'xrt==1.15.0': _VersionConfig('1.15', '1.15.0'),
#     'torch_xla==nightly': _VersionConfig('nightly', 'XRT-dev{}'.format(
#         (datetime.today() - timedelta(1)).strftime('%Y%m%d'))),
# }[VERSION]
# DIST_BUCKET = 'gs://tpu-pytorch/wheels'
# TORCH_WHEEL = 'torch-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
# TORCH_XLA_WHEEL = 'torch_xla-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
# TORCHVISION_WHEEL = 'torchvision-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)

# # Update TPU XRT version
# def update_server_xrt():
#   print('Updating server-side XRT to {} ...'.format(CONFIG.server))
#   url = 'http://{TPU_ADDRESS}:8475/requestversion/{XRT_VERSION}'.format(
#       TPU_ADDRESS=os.environ['COLAB_TPU_ADDR'].split(':')[0],
#       XRT_VERSION=CONFIG.server,
#   )
#   print('Done updating server-side XRT: {}'.format(requests.post(url)))

# update = threading.Thread(target=update_server_xrt)
#update.start()

In [ ]:
import numpy as np
import os
from pathlib import Path
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl
from pytorch_lightning import loggers

In [ ]:
class Discriminator(pl.LightningModule):
  def __init__(self, ndf):
      super().__init__()
      self.ndf = ndf

      self.fc1 = nn.Sequential(
          nn.Conv2d(in_channels=3, out_channels=self.ndf, kernel_size=4, stride=2, padding=1, bias=False),
          nn.LeakyReLU(0.2)
      )
      self.fc2 = nn.Sequential(
          nn.Conv2d(in_channels=self.ndf, out_channels=self.ndf * 2, kernel_size=3, stride=2, padding=1, bias=False),
          nn.LeakyReLU(0.2)
      )
      self.fc3 = nn.Sequential(
          nn.Conv2d(in_channels=self.ndf * 2, out_channels=self.ndf * 4, kernel_size=3, stride=2, padding=1, bias=False),
          nn.LeakyReLU(0.2)
      )
      self.fc4 = nn.Sequential(
          nn.Conv2d(in_channels=self.ndf * 3, out_channels=self.ndf * 8, kernel_size=3, stride=2, padding=1, bias=False),
          torch.nn.Sigmoid()
      )
      self.fc5 = nn.Sequential(
          nn.Conv2d(in_channels=self.ndf * 6, out_channels=1, kernel_size=3, stride=2, padding=1,bias=False),
          nn.Sigmoid()
      )

  def forward(self, x):
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    x = self.fc4(x)
    x = self.fc5(x)

    return x

In [ ]:
class Generator(pl.LightningModule):
  def __init__(self, latent_dim, ngf):
    super().__init__()
    self.ngf = 16
    self.n_features = latent_dim

    self.fc1 = nn.Sequential(
        nn.ConvTranspose2d(in_channels=self.n_features, out_channels=self.ngf * 8, kernel_size=4, stride=1, padding=0, bias=False),
        nn.LeakyReLU(0.2)
    )
    self.fc2 = nn.Sequential(
        nn.ConvTranspose2d(self.ngf * 8, self.ngf * 4, 4, 2, 1, False),
        nn.LeakyReLU(0.2)
    )
    self.fc3 = nn.Sequential(
        nn.ConvTranspose2d(self.ngf * 4, self.ngf * 2, 4, 2, 1, False),
        nn.LeakyReLU(0.2)
    )
    self.fc4 = nn.Sequential(
        nn.ConvTranspose2d(self.ngf * 2, self.ngf, 4, 2, 1, False),
          nn.LeakyReLU(0.2)
    )
    self.fc5 = nn.Sequential(
        nn.ConvTranspose2d(in_channels=self.ngf, out_channels=3, kernel_size=3, stride=1, padding=1, bias=False),
          nn.Tanh()
    )

  def forward(self, x):
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    x = self.fc4(x)
    x = self.fc5(x)

    return x

In [ ]:
class DCGAN(pl.LightningModule):
  def __init__(self, hparams):
    super().__init__()
    self.hparams = hparams

    self.generator = Generator(hparams.latent_dim, hparams.ngf)
    self.discriminator = Discriminator(hparams.ndf)

    # cache for generated images
    self.generated_imgs = None
    self.last_imgs = None

    # For experience replay
    self.exp_replay = torch.tensor([])
     
  def binary_cross_entropy(self, y_hat, y):
    return F.binary_cross_entropy(y_hat, y)

  def forward(self, x):
    return self.generator(x)
  
  def training_step(self, batch, batch_nb, optimizer_idx):
    img, _ = batch
    self.last_imgs = img

    # Train Generator
    if optimizer_idx == 0:
      noise = torch.randn(img.shape[0], self.hparams.latent_dim, 1, 1)

      self.generated_img = self(noise)

      valid_lbl = (0.8 - 1.1) * torch.rand(img.shape[0], 1) + 1.1  # soft labels

      g_loss = self.binary_cross_entropy(self.discriminator(self.generated_img), valid_lbl)
      
      tqdm_dict = {'g_loss': g_loss}
      logs = {"g_loss": g_loss}
      return {"loss": g_loss, "log": logs, 'progress_bar': tqdm_dict}

    # Train Discriminator
    if optimizer_idx == 1:
      valid_lbl = (0.8 - 1.1) * torch.rand(img.shape[0], 1) + 1.1  # soft labels
      unvalid_lbl = (0.0 - 0.3) * torch.rand(img.shape[0], 1) + 0.3  # soft labels

      d_loss = 0

      # Experience replay
      perm = torch.randperm(self.generated_img.size(0)) # Shuffeling
      r_idx = perm[:max(1, self.hparams.experience_save_per_batch)] # Getting the index
      self.exp_replay = torch.cat((self.exp_replay, self.generated_img[r_idx]), 0).detach() # Add our new example to the replay buffer

      if self.exp_replay.size(0) >= self.hparams.experience_batch_size: # when we have enough example from the past train on them
         fake_loss = self.binary_cross_entropy(self.discriminator(self.exp_replay), unvalid_lbl)
         
         d_loss = fake_loss
         self.exp_replay = torch.tensor([])

      else:
        real_loss = self.binary_cross_entropy(self.discriminator(img), valid_lbl)

        fake_loss = self.binary_cross_entropy(self.discriminator(self.generated_img.detach()), unvalid_lbl)
        
        d_loss = (real_loss + fake_loss) / 2

      tqdm_dict = {'d_loss': d_loss}
      logs = {"d_loss": d_loss}
      return {"loss": d_loss, "log": logs, 'progress_bar': tqdm_dict}

  def configure_optimizers(self):
    opt_g = torch.optim.Adam(self.generator.parameters(), lr=self.hparams.lr, betas=(self.hparams.b1, self.hparams.b2))
    opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=self.hparams.lr, betas=(self.hparams.b1, self.hparams.b2))

    return opt_g, opt_d
    
  def train_dataloader(self):
    # transform = transforms.Compose([transforms.Resize((32, 32)),
    #                                 transforms.ToTensor(),
    #                                 transforms.Normalize([0.5], [0.5])])
    # dataset = MNIST(os.getcwd(), train=True, download=True, transform=transform)
    # return DataLoader(dataset,  num_workers=2, batch_size=self.hparams.batch_size)
    transform = transforms.Compose([transforms.Resize((32, 32)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.5], [0.5])
                                    ])

    train_dataset = torchvision.datasets.ImageFolder(
        root="./drive/My Drive/datasets/ghibli_dataset_small_overfit/",
        transform=transform
    )
    return DataLoader(train_dataset, num_workers=3, shuffle=True, batch_size=self.hparams.batch_size)

  def on_epoch_end(self):
      noise = torch.randn(4, self.hparams.latent_dim, 1, 1)

      sample_img = self.generator(noise)
      sample_img = sample_img.view(-1, 1, 32, 32)
      grid = torchvision.utils.make_grid(sample_img, nrow=2).permute(1, 2, 0)
      comet_logger.experiment.log_image(grid, f'generated_images_epoch{self.current_epoch}', step=self.current_epoch)

      if self.current_epoch % 5 == 0:
        trainer.save_checkpoint(checkpoint_folder + "/" + experiment_name + "_epoch_" + str(self.current_epoch) + ".ckpt")
        comet_logger.experiment.log_asset_folder(checkpoint_folder)

        if dirpath.exists() and dirpath.is_dir():
            shutil.rmtree(dirpath)

        # creating checkpoint folder
        access_rights = 0o755
        os.makedirs(checkpoint_folder, access_rights)

In [ ]:
# Parameters
experiment_name =  "GHIBLI_DCGAN_OVERFIT_64px"
dataset_name = "GHIBLI_OVERFIT"
checkpoint_folder = "DCGAN/"
tags = ["DCGAN", "GHIBLI", "OVERFIT", "64x64"]
dirpath = Path(checkpoint_folder )

In [ ]:
# Hyperparameters
from argparse import Namespace

args = {
    'batch_size': 10,
    'lr': 0.0002,
    'b1': 0.5,
    'b2': 0.999,
    'latent_dim': 128,
    'level_of_noise': 0.1,
    'epochs': 10000,
    'r_frequent': 10,
    'ndf': 16,
    'ngf': 16,
    'experience_batch_size': 1000,
    'experience_save_per_batch': 2,
}

hparams = Namespace(**args)

In [ ]:
# init logger
comet_logger = loggers.CometLogger(
    api_key="",
    rest_api_key="",
    project_name="ghibli-gan",
    experiment_name=experiment_name,
)

#defining net
net = DCGAN(hparams)

#logging
comet_logger.experiment.set_model_graph(str(net))
comet_logger.experiment.add_tags(tags=tags)
comet_logger.experiment.log_dataset_info(dataset_name)

INFO:lightning:CometLogger will be initialized in online mode
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/luposx/ghibli-gan/cf125320811b42babf9113ecf1e07015



In [ ]:
# deleting the checkpoint folder
if dirpath.exists() and dirpath.is_dir():
    shutil.rmtree(dirpath)
 
# creating checkpoint folder
access_rights = 0o755
os.makedirs(checkpoint_folder, access_rights)

In [ ]:
# Start training

trainer = pl.Trainer(# resume_from_checkpoint=checkpoint_folder + "/MNIST_GAN_V2_epoch2.ckpt",
                     logger=comet_logger,
                     max_epochs=args["epochs"]
                     )    
trainer.fit(net)
trainer.save_checkpoint(checkpoint_folder + "/" + experiment_name + "_epoch_" + str(args["epochs"]) + ".ckpt")
comet_logger.experiment.log_asset_folder(checkpoint_folder)

INFO:lightning:
   | Name                | Type            | Params
----------------------------------------------------
0  | generator           | Generator       | 434 K 
1  | generator.fc1       | Sequential      | 262 K 
2  | generator.fc1.0     | ConvTranspose2d | 262 K 
3  | generator.fc1.1     | LeakyReLU       | 0     
4  | generator.fc2       | Sequential      | 131 K 
5  | generator.fc2.0     | ConvTranspose2d | 131 K 
6  | generator.fc2.1     | LeakyReLU       | 0     
7  | generator.fc3       | Sequential      | 32 K  
8  | generator.fc3.0     | ConvTranspose2d | 32 K  
9  | generator.fc3.1     | LeakyReLU       | 0     
10 | generator.fc4       | Sequential      | 8 K   
11 | generator.fc4.0     | ConvTranspose2d | 8 K   
12 | generator.fc4.1     | LeakyReLU       | 0     
13 | generator.fc5       | Sequential      | 432   
14 | generator.fc5.0     | ConvTranspose2d | 432   
15 | generator.fc5.1     | Tanh            | 0     
16 | discriminator       | Discriminator   | 79

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/warnings.py:18: RuntimeWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  warnings.warn(*args, **kwargs)


RuntimeError: ignored